#### 위경도변환 그래프생성

In [1]:
import pandas as pd
from geopy.geocoders import Nominatim
import time
from tqdm import tqdm
import numpy as np
print("--- 그래프 생성을 위한 지오코딩 시작 ---")

# ==============================================================================
# 1. 노드 특징 데이터 불러오기
# ==============================================================================
try:
    df = pd.read_csv("node_features.csv")
    print("✅ Step 1: 'node_features.csv' 로딩 성공!")
except FileNotFoundError:
    print("❌ 오류: 'node_features.csv' 파일을 찾을 수 없습니다. 이전 단계를 먼저 실행해주세요.")
    exit()

--- 그래프 생성을 위한 지오코딩 시작 ---
✅ Step 1: 'node_features.csv' 로딩 성공!


In [2]:

# ==============================================================================
# 2. 지오코딩을 통해 주소를 위경도 좌표로 변환
# ==============================================================================
# Nominatim 지오코더 초기화 (공개 서비스이므로 사용 정책 존중 필요)
geolocator = Nominatim(user_agent="MarketQuantum-Project")

# 위도와 경도를 저장할 리스트 생성
latitudes = []
longitudes = []

print("\n--- Step 2: 주소 -> 위경도 변환 작업 시작 (시간이 다소 소요될 수 있습니다) ---")
# tqdm: 진행 상태를 시각적으로 보여주는 라이브러리
for address in tqdm(df['MCT_BSE_AR']):
    try:
        # 지오코딩 실행
        location = geolocator.geocode(address)
        
        if location:
            latitudes.append(location.latitude)
            longitudes.append(location.longitude)
        else:
            latitudes.append(np.nan) # 주소를 찾지 못한 경우 결측치(NaN) 처리
            longitudes.append(np.nan)
            
        # 중요: 공개 API의 과도한 사용을 막기 위해 각 요청 사이에 1초의 지연을 줍니다.
        time.sleep(1)

    except Exception as e:
        print(f"오류 발생: {e}")
        latitudes.append(np.nan)
        longitudes.append(np.nan)

# 변환된 위경도 정보를 데이터프레임에 새로운 컬럼으로 추가
df['latitude'] = latitudes
df['longitude'] = longitudes

print("\n✅ Step 2: 지오코딩 작업 완료!")
print(f"  - 총 {len(df)}개 주소 중 {df['latitude'].notna().sum()}개 변환 성공")


--- Step 2: 주소 -> 위경도 변환 작업 시작 (시간이 다소 소요될 수 있습니다) ---


100%|██████████| 4185/4185 [1:53:13<00:00,  1.62s/it]  



✅ Step 2: 지오코딩 작업 완료!
  - 총 4185개 주소 중 4151개 변환 성공


In [3]:
# ==============================================================================
# 3. 결과 확인 및 저장
# ==============================================================================
print("\n--- 위경도 변환 결과 (상위 5개) ---")
print(df[['ENCODED_MCT', 'MCT_BSE_AR', 'latitude', 'longitude']].head())

# 결측치 개수 확인
print("\n--- 변환 실패(결측치) 개수 ---")
print(df[['latitude', 'longitude']].isnull().sum())

# 다음 단계에서 사용할 수 있도록 위경도 좌표가 추가된 데이터를 저장합니다.
output_path = "nodes_with_coords.csv"
df.to_csv(output_path, index=False, encoding='utf-8-sig')

print(f"\n🎉 모든 작업 완료! 위경도 좌표가 추가된 데이터가 '{output_path}' 파일로 저장되었습니다.")


--- 위경도 변환 결과 (상위 5개) ---
  ENCODED_MCT            MCT_BSE_AR   latitude   longitude
0  000F03E44A   서울특별시 성동구 왕십리로4가길 9  37.545333  127.046493
1  002816BA73    서울 성동구 청계천로10나길 78  37.570230  127.035313
2  003473B465     서울특별시 성동구 서울숲길 55  37.547951  127.040910
3  003AC99735  서울특별시 성동구 용답중앙15길 12  37.564492  127.053856
4  0041E4E5AE   서울특별시 성동구 용답중앙15길 1  37.563956  127.052843

--- 변환 실패(결측치) 개수 ---
latitude     34
longitude    34
dtype: int64

🎉 모든 작업 완료! 위경도 좌표가 추가된 데이터가 'nodes_with_coords.csv' 파일로 저장되었습니다.


##### 엣지리스트

In [5]:
!pip install pandas numpy haversine tqdm

In [1]:
import pandas as pd
import numpy as np
from haversine import haversine, Unit
from itertools import combinations
from tqdm import tqdm

print("--- '엣지 리스트' 생성을 통한 그래프 완성 시작 ---")

# ==============================================================================
# 1. 위경도 좌표 데이터 불러오기
# ==============================================================================
try:
    # 지오코딩을 통해 위경도 좌표가 추가된 파일을 불러옵니다.
    df = pd.read_csv("./nodes_with_coords.csv")
    print("✅ Step 1: 'nodes_with_coords.csv' 로딩 성공!")
except FileNotFoundError:
    print("❌ 오류: 'nodes_with_coords.csv' 파일을 찾을 수 없습니다. 이전 단계를 먼저 실행해주세요.")
    exit()


# ==============================================================================
# 2. 엣지(Edge) 생성을 위한 데이터 정제
# ==============================================================================
# 지오코딩에 실패한 (위경도 값이 없는) 가게는 거리 계산이 불가능하므로 제외합니다.
df_clean = df.dropna(subset=['latitude', 'longitude']).reset_index(drop=True)
print(f"✅ Step 2: 결측치 제거 완료! 총 {len(df_clean)}개의 가게로 거리 계산을 시작합니다.")


# ==============================================================================
# 3. '엣지 리스트' 생성
# ==============================================================================
# 엣지(연결)를 정의할 거리 기준 설정 (단위: 미터)
# 예: 100m 안에 있는 가게들은 서로 '인접'해있다고 정의
DISTANCE_THRESHOLD_METERS = 100

# 연결된 가게 쌍(엣지)을 저장할 리스트
edge_list = []

print(f"\n--- Step 3: {DISTANCE_THRESHOLD_METERS}m 이내 가게들을 연결하는 엣지 리스트 생성 중 ---")

# combinations는 모든 가능한 가게 쌍을 중복 없이 만들어줍니다. (예: (A,B), (A,C), (B,C)...)
# tqdm을 이용해 진행 상황을 확인합니다.
for i, j in tqdm(combinations(df_clean.index, 2), total=len(df_clean)*(len(df_clean)-1)//2):
    
    # 두 가게의 좌표 (위도, 경도)
    coords_1 = (df_clean['latitude'][i], df_clean['longitude'][i])
    coords_2 = (df_clean['latitude'][j], df_clean['longitude'][j])
    
    # haversine 라이브러리를 사용해 두 가게 사이의 거리를 미터(m) 단위로 계산
    distance = haversine(coords_1, coords_2, unit=Unit.METERS)
    
    # 만약 거리가 우리가 설정한 기준보다 가깝다면,
    if distance <= DISTANCE_THRESHOLD_METERS:
        # 엣지 리스트에 두 가게의 고유 ID를 추가합니다.
        # (source: 출발 노드, target: 도착 노드)
        edge_list.append((df_clean['ENCODED_MCT'][i], df_clean['ENCODED_MCT'][j]))

print(f"\n✅ Step 3: 엣지 리스트 생성 완료! 총 {len(edge_list)}개의 연결(엣지)을 찾았습니다.")


# ==============================================================================
# 4. 결과 확인 및 저장
# ==============================================================================
# 생성된 엣지 리스트를 DataFrame으로 변환
df_edges = pd.DataFrame(edge_list, columns=['source', 'target'])

print("\n--- 생성된 엣지 리스트 샘플 (상위 5개) ---")
print(df_edges.head())

# 다음 단계인 VGAE 모델링에서 사용할 수 있도록 엣지 리스트를 파일로 저장합니다.
output_path = "./edge_list.csv"
df_edges.to_csv(output_path, index=False, encoding='utf-8-sig')

print(f"\n🎉 그래프 완성! AI가 학습할 엣지 리스트가 '{output_path}' 파일로 저장되었습니다.")

--- '엣지 리스트' 생성을 통한 그래프 완성 시작 ---
✅ Step 1: 'nodes_with_coords.csv' 로딩 성공!
✅ Step 2: 결측치 제거 완료! 총 4151개의 가게로 거리 계산을 시작합니다.

--- Step 3: 100m 이내 가게들을 연결하는 엣지 리스트 생성 중 ---


100%|██████████| 8613325/8613325 [02:28<00:00, 58121.05it/s]



✅ Step 3: 엣지 리스트 생성 완료! 총 72426개의 연결(엣지)을 찾았습니다.

--- 생성된 엣지 리스트 샘플 (상위 5개) ---
       source      target
0  000F03E44A  01EBAA3D0F
1  000F03E44A  02EECADFE6
2  000F03E44A  1C4665B740
3  000F03E44A  268A868E38
4  000F03E44A  2791AFFA12

🎉 그래프 완성! AI가 학습할 엣지 리스트가 './edge_list.csv' 파일로 저장되었습니다.
